# url: https://qiita.com/nekoumei/items/648726e89d05cba6f432

In [1]:
pip install rdata

In [2]:
import pandas as pd
import statsmodels.api as sm
import plotly.express as px

from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr
pandas2ri.activate()

import rdata

import warnings
warnings.filterwarnings('ignore')

## rdaファイルを読み込む方法

### 1. rpy2を使ってR経由でDataFrameに変換する
#### 事前の環境構築（Ubuntu18.04）
Rのインストールから
```
echo -e "\n## For R package"  | sudo tee -a /etc/apt/sources.list
echo "deb https://cran.rstudio.com/bin/linux/ubuntu $(lsb_release -cs)-cran35/" | sudo tee -a /etc/apt/sources.list
sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys E298A3A825C0D65DFD57CBB651716619E084DAB9
sudo apt update
sudo apt install r-base
sudo apt install libxml2-dev libssl-dev libcurl4-openssl-dev
sudo R
>install.packages("devtools")
>devtools::install_github("itamarcaspi/experimentdatar")

```
参考
```
https://qiita.com/JeJeNeNo/items/43fc95c4710c668e86a2
https://qiita.com/MTNakata/items/129d334cea397a6ec0c3
```

In [3]:
# experimentdatar = importr('experimentdatar')
# vouchers = r['vouchers']

### 2. rdataを使う
https://pypi.org/project/rdata/  
R不要で読み込めるのでこちらの方が良い  
参考： https://qiita.com/nekoumei/items/648726e89d05cba6f432#comment-0ea9751e3f01b27b0adb

In [4]:
# 予めhttps://github.com/itamarcaspi/experimentdatar/blob/master/data/vouchers.rdaからrdaファイルをダウンロードしておく
parsed = rdata.parser.parse_file('/content/vouchers.rda')
converted = rdata.conversion.convert(parsed)
vouchers = converted['vouchers']

## (3) Angrist(2002)のTable 3. bogota 1995の再現

In [5]:
formula_x_base = ['VOUCH0']
formula_x_covariate = ['SVY',  'HSVISIT', 'AGE', 'STRATA1', 'STRATA2', 'STRATA3', 'STRATA4',
                       'STRATA5', 'STRATA6', 'STRATAMS', 'D1993', 'D1995', 'D1997',
                       'DMONTH1', 'DMONTH2', 'DMONTH3', 'DMONTH4', 'DMONTH5', 'DMONTH6', 'DMONTH7', 'DMONTH8',
                       'DMONTH9', 'DMONTH10', 'DMONTH11', 'DMONTH12', 'SEX2']
formula_ys = ['TOTSCYRS','INSCHL','PRSCH_C','USNGSCH','PRSCHA_1','FINISH6','FINISH7','FINISH8','REPT6','REPT','NREPT',
             'MARRIED','HASCHILD','HOURSUM','WORKING3']

In [6]:
def get_VOUCH0_regression_summary(df, formula_x_base=None, formula_x_covariate=None, formula_y=None):
    y = df[formula_y]
    if formula_x_covariate is None:
        X = df[formula_x_base]
    else:
        X = df[formula_x_base + formula_x_covariate]
    X = sm.add_constant(X)
    results = sm.OLS(y, X).fit()
    summary = results.summary().tables[1]
    summary = pd.read_html(summary.as_html(), header=0, index_col=0)[0]
    VOUCH0_summary = summary.loc['VOUCH0']
    if formula_x_covariate is None:
        VOUCH0_summary.name = formula_y + '_base'
    else:
        VOUCH0_summary.name = formula_y + '_covariate'
    return VOUCH0_summary

In [7]:
### bogota 1995のデータを抽出する
regression_data = vouchers[(vouchers.TAB3SMPL == 1) & (vouchers.BOG95SMP == 1)]

### まとめて回帰分析を実行
regression_results = []
for formula_y in formula_ys:
    #　共変量を含まない回帰
    regression_results.append(get_VOUCH0_regression_summary(
        regression_data,
        formula_x_base=formula_x_base,
        formula_x_covariate=None,
        formula_y=formula_y)
        )
    # 共変量を含む回帰
    regression_results.append(get_VOUCH0_regression_summary(
        regression_data,
        formula_x_base=formula_x_base,
        formula_x_covariate=formula_x_covariate,
        formula_y=formula_y)
        )

In [8]:
regression_results = pd.concat(regression_results, axis=1).T

## 通学率と奨学金の利用傾向の可視化(ch2_plot2.html)

In [9]:
### PRSCHA_1, USNGSCHに対するVOUCH0の効果を取り出す
using_voucher_results = regression_results.loc[regression_results.index.str.contains('PRSCHA_1|USNGSCH', regex=True)]
using_voucher_results

,coef,std err,t,P>|t|,[0.025,0.975]
USNGSCH_base,0.5089,0.023,22.107,0.000,0.464,0.554
USNGSCH_covariate,0.5042,0.023,22.007,0.000,0.459,0.549
PRSCHA_1_base,0.0629,0.017,3.731,0.000,0.030,0.096
PRSCHA_1_covariate,0.0574,0.017,3.385,0.001,0.024,0.091


In [10]:
# 取り出した効果をplotly expressで可視化
fig = px.scatter(using_voucher_results, x=using_voucher_results.index, y='coef', error_y='std err',
                title='2.3.3 通学と割引券の利用傾向')
fig.show()

In [11]:
fig.write_html('ch2_plot2-1.html', auto_open=False)

## 留年の傾向を可視化

In [12]:
### PRSCH_C,INSCHL,FINISH6-8,REPTに対するVOUCH0の効果を取り出す
going_private_results = regression_results.loc[
    ['FINISH6_covariate', 'FINISH7_covariate', 'FINISH8_covariate', 'INSCHL_covariate', 'NREPT_covariate', 'PRSCH_C_covariate',
    'REPT_covariate', 'REPT6_covariate']
]
going_private_results

,coef,std err,t,P>|t|,[0.025,0.975]
FINISH6_covariate,0.0229,0.012,1.910,0.056,-0.001,0.047
FINISH7_covariate,0.0307,0.020,1.557,0.120,-0.008,0.070
FINISH8_covariate,0.1002,0.027,3.715,0.000,0.047,0.153
INSCHL_covariate,0.0069,0.020,0.340,0.734,-0.033,0.047
NREPT_covariate,-0.0667,0.028,-2.386,0.017,-0.122,-0.012
PRSCH_C_covariate,0.1533,0.028,5.520,0.000,0.099,0.208
REPT_covariate,-0.0548,0.024,-2.328,0.020,-0.101,-0.009
REPT6_covariate,-0.0594,0.025,-2.417,0.016,-0.108,-0.011


In [13]:
### 取り出した効果をggplotで可視化
fig = px.scatter(going_private_results, x=going_private_results.index, y='coef', error_y='std err',
                title='2.4 留年と進級の傾向')
fig.show()

In [14]:
fig.write_html('ch2_plot2-2.html', auto_open=False)

## (4) Angrist(2002)のTable.4 & 6 bogota 1995の再現

In [15]:
## table4に使うデータを抜き出す
data_tbl4_bog95 = vouchers[(vouchers.BOG95SMP == 1) & (vouchers.TAB3SMPL == 1) & (~vouchers.SCYFNSH.isna())
         & (~vouchers.FINISH6.isna()) & (~vouchers.PRSCHA_1.isna()) & (~vouchers.REPT6.isna())
         & (~vouchers.NREPT.isna()) & (~vouchers.INSCHL.isna()) & (~vouchers.FINISH7.isna())
         & (~vouchers.PRSCH_C.isna()) & (~vouchers.FINISH8.isna()) & (~vouchers.PRSCHA_2.isna())
         & (~vouchers.TOTSCYRS.isna()) & (~vouchers.REPT.isna())][['VOUCH0', 'SVY', 'HSVISIT', 'DJAMUNDI', 'PHONE', 'AGE',
         'STRATA1', 'STRATA2', 'STRATA3', 'STRATA4', 'STRATA5', 'STRATA6', 'STRATAMS', 'DBOGOTA', 'D1993', 'D1995', 'D1997',
         'DMONTH1', 'DMONTH2', 'DMONTH3', 'DMONTH4', 'DMONTH5', 'DMONTH6', 'DMONTH7', 'DMONTH8', 'DMONTH9',
         'DMONTH10', 'DMONTH11', 'DMONTH12', 'SEX_MISS', 'FINISH6', 'FINISH7', 'FINISH8',
         'REPT6', 'REPT', 'NREPT', 'SEX2', 'TOTSCYRS', 'MARRIED', 'HASCHILD',
         'HOURSUM','WORKING3', 'INSCHL','PRSCH_C','USNGSCH','PRSCHA_1']]

### 女子生徒のみのデータでの回帰分析

In [16]:
regression_data = data_tbl4_bog95[data_tbl4_bog95.SEX2 == 0]
### まとめて回帰分析を実行
regression_results = []
for formula_y in formula_ys:
    #　共変量を含まない回帰
    regression_results.append(get_VOUCH0_regression_summary(
        regression_data,
        formula_x_base=formula_x_base,
        formula_x_covariate=None,
        formula_y=formula_y)
        )
    # 共変量を含む回帰
    regression_results.append(get_VOUCH0_regression_summary(
        regression_data,
        formula_x_base=formula_x_base,
        formula_x_covariate=formula_x_covariate,
        formula_y=formula_y)
        )

In [17]:
df_results_female = pd.concat(regression_results, axis=1).T
df_results_female

,coef,std err,t,P>|t|,[0.025,0.975]
TOTSCYRS_base,0.1478,0.067,2.197,0.028,0.016,0.280
TOTSCYRS_covariate,0.0909,0.066,1.381,0.168,-0.038,0.220
INSCHL_base,0.0636,0.030,2.143,0.033,0.005,0.122
INSCHL_covariate,0.0347,0.028,1.237,0.217,-0.020,0.090
PRSCH_C_base,0.1852,0.040,4.664,0.000,0.107,0.263
PRSCH_C_covariate,0.1711,0.039,4.337,0.000,0.094,0.249
USNGSCH_base,0.5505,0.032,17.332,0.000,0.488,0.613
USNGSCH_covariate,0.5436,0.032,16.992,0.000,0.481,0.606
PRSCHA_1_base,0.0408,0.023,1.777,0.076,-0.004,0.086
PRSCHA_1_covariate,0.0229,0.023,1.011,0.312,-0.022,0.067


### 男子生徒のみのデータでの回帰分析

In [18]:
regression_data = data_tbl4_bog95[data_tbl4_bog95.SEX2 == 1]
### まとめて回帰分析を実行
regression_results = []
for formula_y in formula_ys:
    #　共変量を含まない回帰
    regression_results.append(get_VOUCH0_regression_summary(
        regression_data,
        formula_x_base=formula_x_base,
        formula_x_covariate=None,
        formula_y=formula_y)
        )
    # 共変量を含む回帰
    regression_results.append(get_VOUCH0_regression_summary(
        regression_data,
        formula_x_base=formula_x_base,
        formula_x_covariate=formula_x_covariate,
        formula_y=formula_y)
        )

In [19]:
df_results_male = pd.concat(regression_results, axis=1).T
df_results_male

,coef,std err,t,P>|t|,[0.025,0.975]
TOTSCYRS_base,-0.0304,0.080,-0.380,0.704,-0.188,0.127
TOTSCYRS_covariate,-0.0286,0.078,-0.366,0.714,-0.182,0.125
INSCHL_base,-0.0259,0.031,-0.825,0.410,-0.088,0.036
INSCHL_covariate,-0.0195,0.030,-0.658,0.511,-0.078,0.039
PRSCH_C_base,0.1351,0.040,3.350,0.001,0.056,0.214
PRSCH_C_covariate,0.1363,0.040,3.433,0.001,0.058,0.214
USNGSCH_base,0.4677,0.033,14.061,0.000,0.402,0.533
USNGSCH_covariate,0.4677,0.033,14.159,0.000,0.403,0.533
PRSCHA_1_base,0.0852,0.025,3.448,0.001,0.037,0.134
PRSCHA_1_covariate,0.0902,0.025,3.555,0.000,0.040,0.140


In [20]:
# 結果を整形
df_results_female['gender'] = 'female'
df_results_male['gender'] = 'male'

## 通学傾向への分析結果の可視化(ch2_plot3.html)

In [21]:
using_voucher_results_gender = pd.concat([df_results_female, df_results_male], axis=0)
using_voucher_results_gender = using_voucher_results_gender.loc[
    using_voucher_results_gender.index.str.contains('PRSCHA_1_covariate|USNGSCH_covariate', regex=True)]

In [22]:
fig = px.scatter(using_voucher_results_gender, x=using_voucher_results_gender.index, y='coef', facet_row='gender', error_y='std err',
                title='2.5 私立学校への入学と奨学金の利用')
fig.show()

In [23]:
fig.write_html('ch2_plot3.html', auto_open=False)

## 留年と通学年数への分析結果の可視化(ch2_plot4.html)

In [24]:
### PRSCH_C,INSCHL,REPT,TOTSCYRS,FINISHに対する分析結果を抜き出す
going_private_results_gender = pd.concat([df_results_female, df_results_male], axis=0)
going_private_results_gender = going_private_results_gender.loc[
    going_private_results_gender.index.str.contains(
        'FINISH.*covariate|INSCHL_covariate|NREPT_covariate|PRSCH_C_covariate|REPT_covariate|REPT6_covariate|TOTSCYRS_covariate',
        regex=True)]

In [25]:
fig = px.scatter(going_private_results_gender, x=going_private_results_gender.index, y='coef', facet_row='gender', error_y='std err',
                title='2.6 留年と進級の傾向')
fig.show()

In [26]:
fig.write_html('ch2_plot4.html', auto_open=False)

## 労働時間に対する分析結果の可視化(ch2_plot5.html)

In [27]:
### HOURに対する分析結果を抜き出す
working_hour_results_gender = pd.concat([df_results_female, df_results_male], axis=0)
working_hour_results_gender = working_hour_results_gender.loc[
    working_hour_results_gender.index.str.contains(
        'HOURSUM_covariate',
        regex=True)]

In [28]:
fig = px.scatter(working_hour_results_gender, x=working_hour_results_gender.index, y='coef', facet_col='gender', error_y='std err',
                title='2.7 労働時間の傾向')
fig.show()

In [29]:
fig.write_html('ch2_plot5.html', auto_open=False)